In [31]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Dense, ZeroPadding2D, BatchNormalization, Flatten, MaxPooling2D, Conv2D, Activation
from keras.models import Sequential, Model
import numpy as np
import pandas as pd
from PIL import Image
import os
from skimage import io
import uuid
import datetime
import random
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras import initializers
from keras import optimizers
from btdnet import *

In [32]:
# model = btd_model((40, 40, 3))
vgg16_model = VGG19(weights='imagenet', input_shape=(48, 48, 3), include_top = False)
model = Sequential()
for layer in vgg16_model.layers:
    model.add(layer)

model.layers.pop()

for layer in model.layers:
    layer.trainable = True

model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [6]:
X_train_df = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/X_train.csv', header=None).values[:, 1:]
Y_train_df = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/Y_train.csv', header=None).values[:, 1]
X_test_df = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/X_test.csv', header=None).values[:, 1:]
Y_test_df = pd.read_csv('/media/herval/Save/School/THB/Master/Semestre3/Projekt/data/processed/all_with_augmented/Y_test.csv', header=None).values[:, 1]

In [7]:
def convertSavedMatrixToApplicableMatrixOfImages(matrix, old_size, new_size):
    m = matrix.shape[0]

    r = matrix[:, :1600]
    g = matrix[:, 1600:3200]
    b = matrix[:, 3200:4800]

    r = r.reshape((m, old_size[0], old_size[1]))
    g = g.reshape((m, old_size[0], old_size[1]))
    b = b.reshape((m, old_size[0], old_size[1]))

    data = np.zeros((m, old_size[0], old_size[1], 3), 'uint8')
    reshaped_data = np.zeros((m, new_size[0], new_size[1], 3), 'uint8')

    data[..., 0] = r
    data[..., 1] = g
    data[..., 2] = b

    for i in range(m):
        img = Image.fromarray(data[i], 'RGB')
        img = img.resize(new_size, Image.BILINEAR)
        reshaped_data[i] = np.asarray(img)

    return reshaped_data

In [15]:
X_train = convertSavedMatrixToApplicableMatrixOfImages(X_train_df, (40, 40), (48, 48))
X_test = convertSavedMatrixToApplicableMatrixOfImages(X_test_df, (40, 40), (48, 48))
Y_train = Y_train_df.reshape((Y_train_df.size, 1))
Y_test = Y_test_df.reshape((Y_test_df.size, 1))

# X_train = X_train_df.reshape((X_train_df.shape[0], 40, 40, 3))
# X_test = X_test_df.reshape((X_test_df.shape[0], 40, 40, 3))

# for i in range(m):
#     img = Image.fromarray(data[0], 'RGB')
#     img = img.resize(new_size, Image.BILINEAR)
#     reshaped_data[i] = np.asarray(img)

# X_train[:] = np.asarray(X_train[:].resize((48, 48), Image.BILINEAR))

X_train = X_train / 255.
X_test = X_test / 255.

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(33812, 48, 48, 3) (5967, 48, 48, 3) (33812, 1) (5967, 1)


In [34]:
# sgd = optimizers.SGD(lr=0.0005)
model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["accuracy"])

In [35]:
epochs = 200
evals = dict()
evals['id'] = list()
evals['train_acc'] = list()
evals['train_loss'] = list()
evals['test_acc'] = list()
evals['test_loss'] = list()
evals['duration'] = list()
# for epoch in range(epochs):
begin = datetime.datetime.now()
random.seed(100)

hist = model.fit(x = X_train, y = Y_train, epochs=epochs, batch_size=128)

end = datetime.datetime.now()

evals['id'].append(uuid.uuid4())
evals['duration'].append((end - begin).total_seconds())
evals['train_acc'].append(hist.history['acc'][0])
evals['train_loss'].append(hist.history['loss'][0])

preds = model.evaluate(x = X_test, y = Y_test)
evals['test_acc'].append(preds[1])
evals['test_loss'].append(preds[0])
print('Test loss: {}, Test acc: {}'.format(preds[0], preds[1]))
model_json = model.to_json()
with open("./classificators/models/class_model_train_loss-{}_train_acc-{}_test_loss-{}_test_acc-{}_{}{}{}{}{}{}.json".format(hist.history['loss'][0],
                                                                           hist.history['acc'][0],
                                                                           preds[0],
                                                                           preds[1],
                                                                           begin.day,
                                                                           begin.month,
                                                                           begin.year,
                                                                           begin.hour,
                                                                           begin.minute,
                                                                           begin.second), "w") as json_file:
    json_file.write(model_json)
model.save_weights("./classificators/weights/class_weights_train_loss-{}_train_acc-{}_{}{}{}{}{}{}.h5".format(hist.history['loss'][0],
                                                                           hist.history['acc'][0],
                                                                           preds[0],
                                                                           preds[1],
                                                                           begin.day,
                                                                           begin.month,
                                                                           begin.year,
                                                                           begin.hour,
                                                                           begin.minute,
                                                                           begin.second))
ids = np.array([evals['id']]).T
durations = np.array([evals['duration']]).T
train_accs = np.array([evals['train_acc']]).T
train_losses = np.array([evals['train_loss']]).T
test_accs = np.array([evals['test_acc']]).T
test_losses = np.array([evals['test_loss']]).T

perfs = np.concatenate((ids, durations, train_accs, train_losses, test_accs, test_losses), axis = 1)
    
performances = pd.DataFrame(perfs, columns = ['id', 'duration', 'train_acc', 'train_loss', 'test_acc', 'test_loss'])
performances.to_csv('./performances/class_performances.csv', index = False, sep = ',')

Epoch 1/200


ResourceExhaustedError: OOM when allocating tensor with shape[128,48,48,64]
	 [[Node: block1_conv1_11/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_6_0/_2747, block1_conv1_10/kernel/read)]]
	 [[Node: loss_6/mul/_2783 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2083_loss_6/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'block1_conv1_11/convolution', defined at:
  File "/home/herval/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/herval/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/herval/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <mod

In [ ]:
preds = model.evaluate(x = X_test, y = Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
print(model.predict(x = X_test))